---

In [ ]:
#@title Step 1 : Installation

!pip install demucs
!apt install ffmpeg

from google.colab import files
import os
import subprocess
import shutil
import zipfile

In [ ]:
#@title Step 2 : Upload Files

# Create or clear the "inputs" directory
input_directory = "inputs"
if os.path.exists(input_directory):
    shutil.rmtree(input_directory)
os.makedirs(input_directory)

# Upload multiple files to the "inputs" directory
uploaded = files.upload()
for filename in uploaded.keys():
    os.rename(filename, os.path.join(input_directory, filename))

In [ ]:
#@title Step 3: Separate Vocals and Music

# Function to delete folders recursively
def delete_folders(directory):
    """Delete folders recursively."""
    for root, dirs, files in os.walk(directory):
        for dir in dirs:
            shutil.rmtree(os.path.join(root, dir))

# Inform the user about the process
print("Deleting any existing folders in '/content/separated/htdemucs'...")

# Delete any existing folders in "/content/separated/htdemucs"
output_directory = "/content/separated"
if os.path.exists(output_directory):
    delete_folders(output_directory)
    print("Existing folders deleted.")
else:
    os.makedirs(output_directory)
    print("No existing folders found.")

# Run Demucs for each uploaded file
for filename in uploaded.keys():
    # Inform the user about the current file being processed
    print(f"Separating vocals and instrumentals for '{filename}'...")

    # Run Demucs to separate the audio file
    command = f'demucs --two-stems=vocals "{os.path.join(input_directory, filename)}" --out "{output_directory}"'
    subprocess.run(command, shell=True)

# Inform the user about the completion of the process
print("Separation completed successfully.")

# Determine the output directory (Demucs creates a folder named 'separated/{model}/{filename}/')
model_name = 'htdemucs'  # Replace with the model used if different
output_files = []

for filename in uploaded.keys():
    output_dir = os.path.join('separated', model_name, os.path.splitext(os.path.basename(filename))[0])
    output_files.extend([os.path.join(output_dir, f) for f in os.listdir(output_dir)])

In [ ]:
#@title Step 4 : Download Files (.zip)
import zipfile

# Function to compress folders to a zip file
def compress_to_zip(source_dir, zip_filename):
    with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, dirs, files in os.walk(source_dir):
            for file in files:
                file_path = os.path.join(root, file)
                relative_path = os.path.relpath(file_path, source_dir)
                zipf.write(file_path, os.path.join('htdemucs', relative_path))

# Delete "outputs.zip" if it exists
if os.path.exists("outputs.zip"):
    os.remove("outputs.zip")

# Compress folders in "/content/separated/htdemucs" to a zip file
source_dir = "/content/separated/htdemucs"
zip_filename = "outputs.zip"
compress_to_zip(source_dir, zip_filename)

# Download the zip file
files.download(zip_filename)